In [99]:
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
import matplotlib.pyplot as plt
from sklearn import ensemble
from sklearn.pipeline import Pipeline

In [100]:
train = pd.read_csv('../fashionmnist/120000_augmented.csv')
train.isnull().any().sum()

0

In [101]:
from PIL import Image
import numpy as np
import os
import pandas as pd
import cv2

testset = []
dir_name = '../public_test_dataset/data/'
lists = sorted(os.listdir(dir_name))
for idx, img in enumerate(lists):
    image_gray = cv2.imread(dir_name+img, cv2.IMREAD_GRAYSCALE)
    image = np.array(image_gray.reshape((784,)))
    # image = np.array(Image.open(dir_name+img)).reshape((784,))
    image = image.tolist()
    testset.append(image)
testset = np.array(testset)
test = pd.DataFrame(testset)
test = test/255.0

In [102]:
private_set = []
dir_name = '../private_test_dataset/data/'
lists = sorted(os.listdir(dir_name))
for idx, img in enumerate(lists):
    image_gray = cv2.imread(dir_name+img, cv2.IMREAD_GRAYSCALE)
    image = np.array(image_gray.reshape((784,)))
    # image = np.array(Image.open(dir_name+img)).reshape((784,))
    image = image.tolist()
    private_set.append(image)
private_set = np.array(private_set)
private = pd.DataFrame(private_set)
private = private/255.0

In [103]:
print(train.shape,'\n', test.shape,'\n',private.shape)

(120000, 785) 
 (10000, 784) 
 (15000, 784)


In [104]:
df_train = train.copy()
df_test = test.copy()
df_private = private.copy()

In [105]:
X_train= df_train.drop(['label'],axis = 1)
y_train = df_train['label']
X_test = df_test
X_private = df_private
# y_test = df_test.drop(['label'],axis = 1)
# y_label = df_test['label']

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_private = X_private.astype('float32')
X_train /= 255.0
# y_test /=255.0

In [106]:
# X_train과 X_label을 하나의 데이터 프레임으로 합침
df_train = pd.concat([X_train, y_train], axis=1)

# 데이터 프레임을 섞음
df_train = df_train.sample(frac=1, random_state=42)

# 섞인 데이터 프레임에서 훈련 데이터와 레이블을 다시 분리
X_train = df_train.drop(['label'], axis=1)
y_train = df_train['label']

In [107]:
from skimage.feature import hog
def apply_hog(images):
    images = np.array(images)
    images = images.reshape(-1, 28, 28)
    result = []
    for image in images:
        hog_features = hog(image, orientations=9, pixels_per_cell=(4, 4),
                        cells_per_block=(2, 2), block_norm='L2')
        result.append(hog_features)
    return np.array(result)

In [108]:
print(X_train.shape)
print(X_test.shape)
print(X_private.shape)

(120000, 784)
(10000, 784)
(15000, 784)


In [109]:
X_train_ens = np.array(X_train)
X_test_ens = np.array(X_test)
X_private_ens = np.array(X_private)

## XGB

In [110]:
xgb = XGBClassifier(
    n_estimators=500, 
    n_jobs=-1, 
    learning_rate=0.5, 
    max_depth= 5, 
    min_child_weight= 1, 
    reg_lambda =20, 
    seed=0
    )
# xgb = XGBClassifier(use_label_encoder=False,objective="multi:softmax",eval_metric="merror",n_jobs=-1)

In [111]:
pca_xgb = PCA(n_components=500, random_state=42)

In [112]:
# 파이프라인 적용
pipe_xgb = Pipeline([
    ('pca_xgb', pca_xgb),
    ('xgb', xgb)
], verbose=True)

In [113]:
pipe_xgb.fit(X_train_ens, y_train)

[Pipeline] ........... (step 1 of 2) Processing pca_xgb, total=  12.1s
[Pipeline] ............... (step 2 of 2) Processing xgb, total= 5.1min


Pipeline(steps=[('pca_xgb', PCA(n_components=500, random_state=42)),
                ('xgb',
                 XGBClassifier(base_score=None, booster=None, callbacks=None,
                               colsample_bylevel=None, colsample_bynode=None,
                               colsample_bytree=None, device=None,
                               early_stopping_rounds=None,
                               enable_categorical=False, eval_metric=None,
                               feature_types=None, gamma=None, grow_policy=None,
                               importance_type=None,
                               interaction_constraints=None, learning_rate=0.5,
                               max_bin=None, max_cat_threshold=None,
                               max_cat_to_onehot=None, max_delta_step=None,
                               max_depth=5, max_leaves=None, min_child_weight=1,
                               missing=nan, monotone_constraints=None,
                               multi_strategy=None, n_estimators=500, n_jobs=-1,
                               num_parallel_tree=None,
                               objective='multi:softprob', ...))],
         verbose=True)

In [114]:
# only XGB mAP for public test
pred_xgb = pipe_xgb.predict(X_test_ens)

f= open("../testResults/testResult_public_xgb.txt","w+")
for idx, y in enumerate(pred_xgb):
    num_str = str(idx).zfill(5)
    f.write(num_str + " " + str(int(y)) + "\n")
f.close()

In [115]:
import sys
import numpy as np
from sklearn.metrics import auc
from collections import Counter

# testResult_path = sys.argv[1]
# label_path = sys.argv[2]

testResult_path = '../testResults/testResult_public_xgb.txt'
label_path = '../mAP/label.txt'

# pred에 해당하는 testResult.txt 파일 읽어오는 부분입니다.
with open(testResult_path, 'r') as file1:
    preds = file1.readlines()

# 정답에 해당하는 label.txt 파일 읽어오는 부분입니다.
with open(label_path, 'r') as file2:
    labels = file2.readlines()
    

# pred와 label의 클래스값만 리스트로 변환하는 부분입니다.
p = np.array([pred.strip().split()[1] for pred in preds])
l = np.array([label.strip().split()[1] for label in labels])

# pred의 클래스 개수를 count하는 부분입니다.
predict_label_count_dict = Counter(p)
predict_label_count_dict = dict(sorted(predict_label_count_dict.items()))

## mAP 계산하는 부분입니다.
AP = []
num_class = 10

# 모든 클래스에 대해 반복
for c, freq in predict_label_count_dict.items() :
    TP = 0
    FN = 0

    temp_precision = []
    temp_recall = []
    
    for i in range(len(p)):
        # TP, FN 계산
        if l[i] == c and p[i] == c :
            TP += 1
        elif l[i] != c and p[i] == c :
            FN += 1
        
        # preciison, recall 계산            
        if TP+FN != 0: 
            temp_precision.append(TP/(TP+FN))
            temp_recall.append(TP/freq)

    # AP 배열에 클래스 각각의 AP value 저장
    # auc : preciison-recall curve의 면적 구해줌
    # print("TP :", TP)
    # print("FN :", FN)
    # print(temp_precision)
    # print(temp_recall)
    AP.append(auc(temp_recall, temp_precision))

mAP = sum(AP) / num_class

# 각각의 클래스에 대한 AP와 mAP의 Table 출력 부분입니다.
class_name = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']
table = "| {:<13} | {:<13} |\n".format("Class", "AP") + "|---------------|---------------|\n"

for c_name, ap in zip(class_name, AP):
    table += "| {:<13} | {:<13.2f} |\n".format(c_name, ap)

table += "| {:<13} | {:<13.2f} |\n".format("mAP", mAP)
xgb_test_mAP = mAP
print(table)


| Class         | AP            |
|---------------|---------------|
| T-shirt/top   | 0.70          |
| Trouser       | 0.97          |
| Pullover      | 0.71          |
| Dress         | 0.81          |
| Coat          | 0.68          |
| Sandal        | 0.92          |
| Shirt         | 0.54          |
| Sneaker       | 0.87          |
| Bag           | 0.94          |
| Ankle boot    | 0.88          |
| mAP           | 0.80          |



## SVM + HOG

In [86]:
pca = PCA(n_components=250, random_state=43)

In [87]:
optimal_c = 8
svc = SVC(gamma='scale',kernel='rbf',C=optimal_c, random_state=43, n_jobs=-1)

In [94]:
from sklearn.preprocessing import FunctionTransformer
pipe_svm = Pipeline([
    ('hog', FunctionTransformer(apply_hog)),
    ('pca', pca),
    ('svm', svc)
], verbose=True)

In [95]:
pipe_svm.fit(X_train_ens, y_train)

[Pipeline] ............... (step 1 of 3) Processing hog, total=  28.0s
[Pipeline] ............... (step 2 of 3) Processing pca, total=   8.5s
[Pipeline] ............... (step 3 of 3) Processing svm, total=12.7min


Pipeline(steps=[('hog',
                 FunctionTransformer(func=<function apply_hog at 0x15e09f8b0>)),
                ('pca', PCA(n_components=250, random_state=43)),
                ('svm', SVC(C=8, random_state=43))],
         verbose=True)

In [96]:
# only XGB mAP for public test
pred_svm = pipe_svm.predict(X_test_ens)

f= open("../testResults/testResult_public_svm.txt","w+")
for idx, y in enumerate(pred_svm):
    num_str = str(idx).zfill(5)
    f.write(num_str + " " + str(int(y)) + "\n")
f.close()

In [97]:
import sys
import numpy as np
from sklearn.metrics import auc
from collections import Counter

# testResult_path = sys.argv[1]
# label_path = sys.argv[2]

testResult_path = '../testResults/testResult_public_svm.txt'
label_path = '../mAP/label.txt'

# pred에 해당하는 testResult.txt 파일 읽어오는 부분입니다.
with open(testResult_path, 'r') as file1:
    preds = file1.readlines()

# 정답에 해당하는 label.txt 파일 읽어오는 부분입니다.
with open(label_path, 'r') as file2:
    labels = file2.readlines()
    

# pred와 label의 클래스값만 리스트로 변환하는 부분입니다.
p = np.array([pred.strip().split()[1] for pred in preds])
l = np.array([label.strip().split()[1] for label in labels])

# pred의 클래스 개수를 count하는 부분입니다.
predict_label_count_dict = Counter(p)
predict_label_count_dict = dict(sorted(predict_label_count_dict.items()))

## mAP 계산하는 부분입니다.
AP = []
num_class = 10

# 모든 클래스에 대해 반복
for c, freq in predict_label_count_dict.items() :
    TP = 0
    FN = 0

    temp_precision = []
    temp_recall = []
    
    for i in range(len(p)):
        # TP, FN 계산
        if l[i] == c and p[i] == c :
            TP += 1
        elif l[i] != c and p[i] == c :
            FN += 1
        
        # preciison, recall 계산            
        if TP+FN != 0: 
            temp_precision.append(TP/(TP+FN))
            temp_recall.append(TP/freq)

    # AP 배열에 클래스 각각의 AP value 저장
    # auc : preciison-recall curve의 면적 구해줌
    # print("TP :", TP)
    # print("FN :", FN)
    # print(temp_precision)
    # print(temp_recall)
    AP.append(auc(temp_recall, temp_precision))

mAP = sum(AP) / num_class

# 각각의 클래스에 대한 AP와 mAP의 Table 출력 부분입니다.
class_name = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']
table = "| {:<13} | {:<13} |\n".format("Class", "AP") + "|---------------|---------------|\n"

for c_name, ap in zip(class_name, AP):
    table += "| {:<13} | {:<13.2f} |\n".format(c_name, ap)

table += "| {:<13} | {:<13.2f} |\n".format("mAP", mAP)
svm_test_mAP = mAP
print(table)


| Class         | AP            |
|---------------|---------------|
| T-shirt/top   | 0.76          |
| Trouser       | 0.98          |
| Pullover      | 0.81          |
| Dress         | 0.84          |
| Coat          | 0.74          |
| Sandal        | 0.97          |
| Shirt         | 0.60          |
| Sneaker       | 0.90          |
| Bag           | 0.98          |
| Ankle boot    | 0.95          |
| mAP           | 0.85          |



In [98]:
# model 저장
import pickle
with open('../models/svm_hog_pipeline', 'wb') as f:
    pickle.dump(pipe_svm, f)

## 앙상블

In [ ]:
models = list()
models.append(('xgb', pipe_xgb))
models.append(('svm_hog', pipe_svm))

In [ ]:
from sklearn.ensemble import VotingClassifier
# define the voting ensemble
voting = VotingClassifier(estimators=models, voting='soft')

In [ ]:
voting.fit(X_train_ens, y_train)

In [ ]:
pred_voting = voting.predict(X_test_ens)
f= open("../testResults/testResult_public_voting.txt","w+")
for idx, y in enumerate(pred_voting):
    num_str = str(idx).zfill(5)
    f.write(num_str + " " + str(int(y)) + "\n")
f.close()

In [ ]:
import sys
import numpy as np
from sklearn.metrics import auc
from collections import Counter

# testResult_path = sys.argv[1]
# label_path = sys.argv[2]

testResult_path = '../testResults/testResult_public_voting.txt'
label_path = '../mAP/label.txt'

# pred에 해당하는 testResult.txt 파일 읽어오는 부분입니다.
with open(testResult_path, 'r') as file1:
    preds = file1.readlines()

# 정답에 해당하는 label.txt 파일 읽어오는 부분입니다.
with open(label_path, 'r') as file2:
    labels = file2.readlines()
    

# pred와 label의 클래스값만 리스트로 변환하는 부분입니다.
p = np.array([pred.strip().split()[1] for pred in preds])
l = np.array([label.strip().split()[1] for label in labels])

# pred의 클래스 개수를 count하는 부분입니다.
predict_label_count_dict = Counter(p)
predict_label_count_dict = dict(sorted(predict_label_count_dict.items()))

## mAP 계산하는 부분입니다.
AP = []
num_class = 10

# 모든 클래스에 대해 반복
for c, freq in predict_label_count_dict.items() :
    TP = 0
    FN = 0

    temp_precision = []
    temp_recall = []
    
    for i in range(len(p)):
        # TP, FN 계산
        if l[i] == c and p[i] == c :
            TP += 1
        elif l[i] != c and p[i] == c :
            FN += 1
        
        # preciison, recall 계산            
        if TP+FN != 0: 
            temp_precision.append(TP/(TP+FN))
            temp_recall.append(TP/freq)

    # AP 배열에 클래스 각각의 AP value 저장
    # auc : preciison-recall curve의 면적 구해줌
    # print("TP :", TP)
    # print("FN :", FN)
    # print(temp_precision)
    # print(temp_recall)
    AP.append(auc(temp_recall, temp_precision))

mAP = sum(AP) / num_class

# 각각의 클래스에 대한 AP와 mAP의 Table 출력 부분입니다.
class_name = ['T-shirt/top','Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']
table = "| {:<13} | {:<13} |\n".format("Class", "AP") + "|---------------|---------------|\n"

for c_name, ap in zip(class_name, AP):
    table += "| {:<13} | {:<13.2f} |\n".format(c_name, ap)

table += "| {:<13} | {:<13.2f} |\n".format("mAP", mAP)
voting_test_mAP = mAP
print(table)
